In [154]:
!pip install anvil-uplink
import anvil.server

anvil.server.connect("server_AWZDTJC7ELLLWKQJM7XSDOUT-QMTVJN4LDJALI4EQ")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [155]:
# constansts

START_WORD_LENGTHS = [4,5,6]

MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 9

NUM_STEPS = 8
MIN_STEPS = 5


In [156]:
'''
Created on Feb 7, 2013

@author: olegs
'''

ROMAN_CONSTANTS = (
            ( "", "I", "II", "III", "IV", "V", "VI", "VII", "VIII", "IX" ),
            ( "", "X", "XX", "XXX", "XL", "L", "LX", "LXX", "LXXX", "XC" ),
            ( "", "C", "CC", "CCC", "CD", "D", "DC", "DCC", "DCCC", "CM" ),
            ( "", "M", "MM", "MMM", "",   "",  "-",  "",    "",     ""   ),
        )

ROMAN_SYMBOL_MAP = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)

CUTOFF = 4000
BIG_DEC = 2900
BIG_ROMAN = "MMCM"
ROMAN_NOUGHT = "nulla"

def digits(num):
    if num < 0:
        raise Exception('range error: negative numbers not supported')
    if num % 1 != 0.0:
        raise Exception('floating point numbers not supported')
    res = []
    while num > 0:
        res.append(num % 10)
        num //= 10
    return res

def toString(num, emptyZero=False):
    if num < CUTOFF:
        digitlist = digits(num)
        if digitlist:
            res = reversed([ ROMAN_CONSTANTS[order][digit] for order, digit in enumerate(digitlist) ])
            return "".join(res)
        else:
            return "" if emptyZero else ROMAN_NOUGHT 
    else:
        if num % 1 != 0.0:
            raise Exception('floating point numbers not supported')
        # For numbers over or equal the CUTOFF, the remainder of division by 2900
        # is represented as above, prepended with the multiples of MMCM (2900 in Roman),
        # which guarantees no more than 3 repetitive Ms.
        return BIG_ROMAN * (num // BIG_DEC) + toString(num % BIG_DEC, emptyZero=True)

def parse(numeral):
    numeral = numeral.upper()
    result = 0

    lastVal = 0
    lastCount = 0
    subtraction = False
    for symbol in numeral[::-1]:
        value = ROMAN_SYMBOL_MAP.get(symbol)
        if not value:
            return False
        if lastVal == 0:
            lastCount = 1
            lastVal = value
        elif lastVal == value:
            lastCount += 1
            # exceptions
        else:
            result += (-1 if subtraction else 1) * lastVal * lastCount
            subtraction = lastVal > value
            lastCount = 1
            lastVal = value
    
    result = result + (-1 if subtraction else 1) * lastVal * lastCount
    return True

In [157]:
import requests
import time

dictionaryFileName = 'assets/dictionary_FULL_IT.txt'
dictionaryFileName = 'assets/Ita.txt'

    
def normalizeWord(word):
    return word.strip().upper()
    
LOAD_FROM_URL = True

t0 = time.time()

lines = []
if LOAD_FROM_URL:
    baseUrl = 'https://azrafe7.github.io/'
    f = requests.get(baseUrl + dictionaryFileName)
    lines = f.text.splitlines()
else:
    with open(dictionaryFileName) as f:
        lines = f.read().splitlines()

filteredWords = []
for line in lines:
    word = normalizeWord(line)
    length = len(word)
    if length >= MIN_WORD_LENGTH and length <= MAX_WORD_LENGTH and not parse(word):
        filteredWords.append(word)

print("Dictionary loaded in {0:.2f}s".format(time.time() - t0))

print('{0}/{1}'.format(len(filteredWords), len(lines)))

print(filteredWords[:10])

Dictionary loaded in 1.66s
401123/710993
['AALENIANA', 'AALENIANE', 'AALENIANI', 'AALENIANO', 'ABABUA', 'ABACA', 'ABACETI', 'ABACETO', 'ABACHI', 'ABACHISTA']


In [158]:

def wordsOfLength(lengths):
    return [word for word in filteredWords if len(word) in lengths]

len(list(wordsOfLength(START_WORD_LENGTHS)))


35094

In [159]:

def isAnagram(sourceWord, targetWord):
    if len(sourceWord) != len(targetWord):
        return False
    
    res = True
    
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    #print(targetCharSet)
    for i, ch in enumerate(sourceWord):
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
        else:
            res = False
            break
    
    return res
    
        
assert(isAnagram('', ''))
assert(not isAnagram('corto', 'trOca'))
assert(not isAnagram('c', ''))
assert(isAnagram('cortao', 'trocao'))

srcWord = 'ostra'
dstWord ='motri'

print('isAnagram("{0}", "{1}")'.format(srcWord, dstWord))
print(isAnagram(srcWord, dstWord))

isAnagram("ostra", "motri")
False


In [160]:
import time

def isAnagramEx(sourceWord, targetWord, maxDiffLength=0, strict=False):
    if maxDiffLength == 0:
        return isAnagram(sourceWord, targetWord)
    else:
        wordDiffLength = abs(len(sourceWord) - len(targetWord))
        if wordDiffLength > maxDiffLength:
            return False
        
    sourceCharSet = list(sourceWord)
    targetCharSet = list(targetWord)
    
    if len(sourceCharSet) > len(targetCharSet):
        sourceCharSet, targetCharSet = targetCharSet, sourceCharSet
        
    #print(targetCharSet, sourceCharSet)
    for ch in sourceCharSet:
        #print(ch, targetCharSet)
        if ch in targetCharSet:
            targetCharSet.remove(ch)
    
    #print(targetCharSet)
    targetCharSetLength = len(targetCharSet)
    #print(wordDiffLength, targetCharSet)
    isLooseAnagram = (targetCharSetLength <= maxDiffLength)
    res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and wordDiffLength == maxDiffLength)
    return res
    
        
maxDiffLength=2

assert(isAnagramEx('', ''))
assert(not isAnagramEx('corto', 'trOca'))
assert(not isAnagramEx('c', ''))
assert(isAnagramEx('cortao', 'trocao'))
assert(isAnagramEx('cortao', 'trocaox', 1))
assert(not isAnagramEx('strappo', 'posta', 1))
assert(isAnagramEx('strappo', 'posta', 2))
assert(isAnagramEx('strappo', 'posta', 3))
assert(not isAnagramEx('strappo', 'posta', 3, strict=True))
assert(isAnagramEx('strappo', 'posta', 2, strict=True))
assert(isAnagramEx('LEVI', 'ABILE', 2,))
assert(not isAnagramEx('TUFFAI', 'ABACA', 2,))


srcWord = 'straccio'
dstWord ='crosta'

print('isAnagramEx("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(5000):
  isAnagramEx('', '')
  not isAnagramEx('corto', 'trOca')
  not isAnagramEx('c', '')
  isAnagramEx('cortao', 'trocao')
  isAnagramEx('cortao', 'trocaox', 1)
  not isAnagramEx('strappo', 'posta', 1)
  isAnagramEx('strappo', 'posta', 2)
  isAnagramEx('strappo', 'posta', 3)
  not isAnagramEx('strappo', 'posta', 3, strict=True)
  isAnagramEx('strappo', 'posta', 2, strict=True)
  isAnagramEx('LEVI', 'ABILE', 2, )
  not isAnagramEx('TUFFAI', 'ABACA', 2, )  
print(f'{(time.time() - t0):.4f}s')


isAnagramEx("straccio", "crosta", 2)
True
0.0769s


In [161]:
import time


def _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=0, strict=False):
  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:
  #   wordDiffLength = abs(sourceLength - targetLength)
  #   if wordDiffLength > maxDiffLength:
  #       return False

  i = 0
  j = 0
  diff = 0
  same = 0
  if sourceLength != 0 and targetLength != 0:
    while i < sourceLength and j < targetLength:
      sourceCh = sortedSourceChars[i]
      targetCh = sortedTargetChars[j]
      if sourceCh == targetCh:
        i += 1
        j += 1
        same += 1
      else:
        if sourceCh < targetCh:
          i += 1
        else:
          j += 1

  diff = max(sourceLength, targetLength) - same
  #print(sortedSourceChars, sortedTargetChars, diff, same)
  isLooseAnagram = diff <= maxDiffLength
  res = (not strict and isLooseAnagram) or (strict and isLooseAnagram and diff == maxDiffLength)
  return res, diff


def isAnagramEx2(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sortedSourceChars = sorted(sourceWord)
  sortedTargetChars = sorted(targetWord)

  # if maxDiffLength == 0:
  #     return sortedSourceChars == sortedTargetChars
  # else:


  sourceLength = len(sortedSourceChars)
  targetLength = len(sortedTargetChars)

  return _isAnagramEx2(sortedSourceChars, sortedTargetChars, sourceLength, targetLength, maxDiffLength=maxDiffLength, strict=strict)

maxDiffLength = 2

assert (True, 0) == isAnagramEx2('', '')
assert (False, 2) == isAnagramEx2('corto', 'trOca')
assert (False, 4) == isAnagramEx2('corto', 'trOcaul')
assert (False, 1) == isAnagramEx2('c', '')
assert (True, 1) == isAnagramEx2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagramEx2('cortao', 'trocao')
assert (False, 2) == isAnagramEx2('strappo', 'posta', 1)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 3)
assert (False, 2) == isAnagramEx2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagramEx2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagramEx2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagramEx2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagramEx2('AOPST', 'AOPSTU', 1, strict=True)

srcWord = 'straccio'
dstWord = 'crosta'

print('isAnagramEx2("{0}", "{1}", {2})'.format(srcWord, dstWord, maxDiffLength))
print(isAnagramEx2(srcWord, dstWord, maxDiffLength=maxDiffLength))

t0 = time.time()
for i in range(5000):
  isAnagramEx2('', '')
  not isAnagramEx2('corto', 'trOca')
  not isAnagramEx2('c', '')
  isAnagramEx2('cortao', 'trocao')
  isAnagramEx2('cortao', 'trocaox', 1)
  not isAnagramEx2('strappo', 'posta', 1)
  isAnagramEx2('strappo', 'posta', 2)
  isAnagramEx2('strappo', 'posta', 3)
  not isAnagramEx2('strappo', 'posta', 3, strict=True)
  isAnagramEx2('strappo', 'posta', 2, strict=True)
  isAnagramEx2('LEVI', 'ABILE', 2, )
  not isAnagramEx2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


isAnagramEx2("straccio", "crosta", 2)
(True, 2)
0.1522s


In [162]:
import time
from collections import Counter

def isAnagram_Counter(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter(sourceWord)
  targetCounter = Counter(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter('corto', 'trOca'))


assert (True, 0) == isAnagram_Counter('', '')
assert (False, 2) == isAnagram_Counter('corto', 'trOca')
assert (False, 4) == isAnagram_Counter('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter('c', '')
assert (True, 1) == isAnagram_Counter('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter('BAVA', 'ABAVI', 1, strict=True)

t0 = time.time()
for i in range(5000):
  isAnagram_Counter('', '')
  not isAnagram_Counter('corto', 'trOca')
  not isAnagram_Counter('c', '')
  isAnagram_Counter('cortao', 'trocao')
  isAnagram_Counter('cortao', 'trocaox', 1)
  not isAnagram_Counter('strappo', 'posta', 1)
  isAnagram_Counter('strappo', 'posta', 2)
  isAnagram_Counter('strappo', 'posta', 3)
  not isAnagram_Counter('strappo', 'posta', 3, strict=True)
  isAnagram_Counter('strappo', 'posta', 2, strict=True)
  isAnagram_Counter('LEVI', 'ABILE', 2, )
  not isAnagram_Counter('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


-> (False, 2)
0.8381s


In [163]:
import time
from copy import copy

class Counter2(dict):
  
  def __init__(self, word):
    for ch in word:
      if ch not in self:
        self[ch] = 0
      self[ch] += 1
      # if ch not in self:
      #   self[ch] = word.count(ch)

  def __add__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      newCounter[k] += v
    
    return newCounter

  def __sub__(self, otherCounter):
    newCounter = copy(self)
    for k, v in otherCounter.items():
      if k not in newCounter:
        newCounter[k] = 0
      else:
        newCounter[k] -= v
        if newCounter[k] <= 0:
          del newCounter[k]
    
    return newCounter

  def isAnagram(self, otherCounter, maxDiffLength=1, strict=False):
    sourceCounter, targetCounter = self, otherCounter
    if len(sourceCounter) < len(targetCounter):
      sourceCounter, targetCounter = targetCounter, sourceCounter
    diffCounter = sourceCounter - targetCounter
    diffSum = sum(diffCounter.values())
   
    isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
    return isAnagram, diffSum


def isAnagram_Counter2(sourceWord, targetWord, maxDiffLength=0, strict=False):
  sourceCounter = Counter2(sourceWord)
  targetCounter = Counter2(targetWord)

  # swap? (lengthier first!)
  if len(sourceWord) < len(targetWord):
    sourceCounter, targetCounter = targetCounter, sourceCounter

  diffCounter = sourceCounter - targetCounter

  diffSum = sum(diffCounter.values())

  isAnagram = (not strict and diffSum <= maxDiffLength) or (strict and diffSum == maxDiffLength)
  return isAnagram, diffSum

print("->", isAnagram_Counter2('corto', 'trOca'))

word1 = Counter2('corto')
word2 = Counter2('trOca')
display(word1)
display(word2)
display(word1 + word2)
display(word1 - word2)
display(Counter('corto') - Counter('trOca'))

assert (True, 0) == isAnagram_Counter2('', '')
assert (False, 2) == isAnagram_Counter2('corto', 'trOca')
assert (False, 4) == isAnagram_Counter2('corto', 'trOcaul')
assert (False, 1) == isAnagram_Counter2('c', '')
assert (True, 1) == isAnagram_Counter2('cortao', 'trocaox', 1)
assert (True, 0) == isAnagram_Counter2('cortao', 'trocao')
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 1)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 3)
assert (False, 2) == isAnagram_Counter2('strappo', 'posta', 3, strict=True)
assert (True, 2) == isAnagram_Counter2('strappo', 'posta', 2, strict=True)
assert (True, 2) == isAnagram_Counter2('LEVI', 'ABILE', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=False)
assert (False, 5) == isAnagram_Counter2('TUFFAI', 'ABACA', 2, strict=True)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=False)
assert (True, 1) == isAnagram_Counter2('AOPST', 'AOPSTU', 1, strict=True)
assert (True, 1) == isAnagram_Counter2('BAVA', 'ABAVI', 1, strict=True)
assert (True, 1) == Counter2('BAVA').isAnagram(Counter2('ABAVI'), maxDiffLength=1)

t0 = time.time()
for i in range(5000):
  isAnagram_Counter2('', '')
  not isAnagram_Counter2('corto', 'trOca')
  not isAnagram_Counter2('c', '')
  isAnagram_Counter2('cortao', 'trocao')
  isAnagram_Counter2('cortao', 'trocaox', 1)
  not isAnagram_Counter2('strappo', 'posta', 1)
  isAnagram_Counter2('strappo', 'posta', 2)
  isAnagram_Counter2('strappo', 'posta', 3)
  not isAnagram_Counter2('strappo', 'posta', 3, strict=True)
  isAnagram_Counter2('strappo', 'posta', 2, strict=True)
  isAnagram_Counter2('LEVI', 'ABILE', 2, )
  not isAnagram_Counter2('TUFFAI', 'ABACA', 2, )
print(f'{(time.time() - t0):.4f}s')


-> (False, 2)


{'c': 1, 'o': 2, 'r': 1, 't': 1}

{'t': 1, 'r': 1, 'O': 1, 'c': 1, 'a': 1}

{'c': 2, 'o': 2, 'r': 2, 't': 2, 'O': 1, 'a': 1}

{'o': 2, 'O': 0, 'a': 0}

Counter({'o': 2})

0.7616s


In [164]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
NodeCount = 0
WordCount = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class TrieNode:
    def __init__(self):
        self.word = None
        self.children = {}

        global NodeCount
        NodeCount += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children: 
                node.children[letter] = TrieNode()

            node = node.children[letter]

        node.word = word

# read dictionary file into a trie
start = time.time()

trie = TrieNode()
for word in filteredWords:
    WordCount += 1
    trie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print("Read %d words into %d nodes" % (WordCount, NodeCount))

# The search function returns a list of all words that are less than the given
# maximum distance from the target word
def search( word, maxCost ):

    # build first row
    currentRow = range( len(word) + 1 )

    results = []

    # recursively search each branch of the trie
    for letter in trie.children:
        searchRecursive( trie.children[letter], letter, word, currentRow, 
            results, maxCost )

    return results

# This recursive helper is used by the search function above. It assumes that
# the previousRow has been filled in already.
def searchRecursive( node, letter, word, previousRow, results, maxCost ):

    columns = len( word ) + 1
    currentRow = [ previousRow[0] + 1 ]

    # Build one row for the letter, with a column for each letter in the target
    # word, plus one for the empty string at column 0
    for column in range( 1, columns ):

        insertCost = currentRow[column - 1] + 1
        deleteCost = previousRow[column] + 1

        if word[column - 1] != letter:
            replaceCost = previousRow[ column - 1 ] + 1
        else:                
            replaceCost = previousRow[ column - 1 ]

        currentRow.append( min( insertCost, deleteCost, replaceCost ) )

    # if the last entry in the row indicates the optimal cost is less than the
    # maximum cost, and there is a word in this trie node, then add it.
    if currentRow[-1] <= maxCost and node.word != None:
        results.append( (node.word, currentRow[-1] ) )

    # if any entries in the row are less than the maximum cost, then 
    # recursively search each branch of the trie
    if min( currentRow ) <= maxCost:
        for letter in node.children:
            searchRecursive( node.children[letter], letter, word, currentRow, 
                results, maxCost )

start = time.time()
results = search( TARGET, MAX_COST )
end = time.time()

#for result in results: print(result)

print("Search took %g s" % (end - start))



Build took 10.7111 s
Read 401123 words into 674739 nodes
Search took 2.89868 s


In [165]:
# http://stevehanov.ca/blog/index.php?id=114
# By Steve Hanov, 2011. Released to the public domain
import time
import sys

DICTIONARY = "/usr/share/dict/words";
TARGET = "SCORTA"
MAX_COST = 4

# Keep some interesting statistics
CounterNodes = 0

# The Trie data structure keeps a set of words, organized with one node for
# each letter. Each node has a branch for each letter that may follow it in the
# set of words.
class CounterNode:
    def __init__(self):
        self.word = None
        self.count = None
        self.children = {}

        global CounterNodes
        CounterNodes += 1

    def insert( self, word ):
        node = self
        for letter in word:
            if letter not in node.children: 
                node.children[letter] = CounterNode()

            node = node.children[letter]
            node.count = word.count(letter)

        node.word = word

    def __repr__(self):
      return f'{self.word} {self.children} {self.count}'

# read dictionary file into a trie
start = time.time()

counterTrie = CounterNode()
for word in ['carta', 'corta', 'bara', 'cara', 'branco', 'nostro']:
    counterTrie.insert( word )

end = time.time()
print("Build took %g s" % (end - start))

print(counterTrie)
counterTrie.children['c'].children['a'].count


Build took 0.000504732 s
None {'c': None {'a': None {'r': None {'t': None {'a': carta {} 2} 1, 'a': cara {} 2} 1} 2, 'o': None {'r': None {'t': None {'a': corta {} 1} 1} 1} 1} 1, 'b': None {'a': None {'r': None {'a': bara {} 2} 1} 2, 'r': None {'a': None {'n': None {'c': None {'o': branco {} 1} 1} 1} 1} 1} 1, 'n': None {'o': None {'s': None {'t': None {'r': None {'o': nostro {} 2} 1} 1} 1} 2} 1} None


2

In [166]:
import random

startWord = random.choice(filteredWords)
print("START WORD:", startWord)

start = time.time()
results = search(startWord, 3)
end = time.time()

print("Search took %g s" % (end - start))

validEndWord = random.choice(results)

print("END WORD:", validEndWord)


START WORD: ABOLITOTI
Search took 1.24377 s
END WORD: ('ABOLITETI', 1)


In [167]:
def findAnagramsEx(wordList, word, maxDiffLength=1, strict=False):
    results = []
    wordLength = len(word)
    for entry in wordList:
        entryLength = len(entry)
        if word == entry or entryLength < wordLength - maxDiffLength or entryLength > wordLength + maxDiffLength:
            continue
        if isAnagramEx(entry, word, maxDiffLength=maxDiffLength, strict=strict):
            diffLength = abs(len(entry) - len(word))
            results.append((entry, diffLength))
    
    return results


In [168]:
import random

newStartWord = random.choice(filteredWords)
print("START WORD:", startWord)

anagramsDict = {0:[], 1:[], 2:[]}
randomFilteredWords = filteredWords[:]
random.shuffle(filteredWords[:])
randomFilteredWords = sorted(randomFilteredWords)

anagrams = findAnagramsEx(randomFilteredWords, newStartWord, maxDiffLength=1, strict=False)
for anagram in anagrams:
    anagramsDict[anagram[1]].append(anagram[0])
    
print(anagrams)
#print(anagramsDict[0][:10])

START WORD: ABOLITOTI
[('ERIOFILI', 0), ('ERIOFORI', 0), ('FERITORI', 0), ('FIORERAI', 0), ('FIOREREI', 0), ('FIORERIA', 0), ('FIORERIE', 0), ('FIORIERA', 0), ('FIORIERE', 0), ('FIORIFERI', 1), ('FIORIRAI', 0), ('FIORIRCI', 0), ('FIORIRE', 1), ('FIORIRLE', 0), ('FIORIRLI', 0), ('FIORIRMI', 0), ('FIORIRNE', 0), ('FIORIRSI', 0), ('FIORIRTI', 0), ('FIORIRVI', 0), ('FORBIREI', 0), ('FORIERI', 1), ('FORNIREI', 0), ('FORZIERI', 0), ('FRIGORIE', 0), ('FRINIREI', 0), ('INFERIORI', 1), ('INFERIRO', 0), ('INFERRIO', 0), ('INFIERIR', 0), ('INFIERIRO', 1), ('IODIFERI', 0), ('OFFRIREI', 0), ('ORRIFICI', 0), ('PORFIREI', 0), ('PORFIRIE', 0), ('PORIFERI', 0), ('PREFIORI', 0), ('PREFIORII', 1), ('PROFERII', 0), ('REIFORMI', 0), ('RIFEDIRO', 0), ('RIFERII', 1), ('RIFERIRO', 0), ('RIFERITI', 0), ('RIFERITO', 0), ('RIFERIVI', 0), ('RIFERIVO', 0), ('RIFIDERO', 0), ('RIFILERO', 0), ('RIFINERO', 0), ('RIFINIRE', 0), ('RIFINIRO', 0), ('RIFIORI', 1), ('RIFIORII', 0), ('RIFIORIR', 0), ('RIFIORIRE', 1), ('RIFIO

In [169]:
print(anagramsDict[1])

['FIORIFERI', 'FIORIRE', 'FORIERI', 'INFERIORI', 'INFIERIRO', 'PREFIORII', 'RIFERII', 'RIFIORI', 'RIFIORIRE', 'RIFIORITE', 'SFIORIREI']


In [170]:
import time

def generateChain(quiet=False, startWord=None, strict=False):
    startWords = wordsOfLength(START_WORD_LENGTHS)
    
    restart = True
    
    while restart:
        restart = False
        startWord = random.choice(startWords)
        print("START WORD:", startWord)
    
        steps = [startWord]
    
        prevStep = (startWord, 0)
        #self.info.text = ""
        for i in range(6):
            choice = random.choice(["anagram", "levenshtein"])
            if choice == "anagram":
                if not quiet: print("Finding anagrams...", end="")
                startTime = time.time()
                anagrams = findAnagramsEx(filteredWords, prevStep[0], maxDiffLength=1, strict=strict)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(anagrams) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(anagrams))
                    break
            else:
                if not quiet: print("Finding levenshtein...", end="")
                startTime = time.time()
                results = search(prevStep[0], 1)
                if not quiet: print(" ({0:.2f})".format(time.time() - startTime))
                if len(results) < 10 and i < MIN_STEPS:
                    restart = True
                    print(choice, len(results))
                    break
                    
            done = False
            while not done:
                if choice == "anagram":
                    nextStep = random.choice(anagrams)
                else:
                    nextStep = random.choice(results)
    
                if nextStep[0] not in steps:
                    steps.append(nextStep[0])
                    done = True
    
            resultsCount = len(results) if choice == "levenshtein" else len(anagrams)
            if not quiet: print(nextStep, resultsCount, choice)
            prevStep = nextStep
    
        print("restart", restart)
        print()
    
    endWord = steps[-1]
    print("END WORD:", endWord)  
    return steps

t0 = time.time()
steps = generateChain(startWord=None, strict=True)
print("Elapsed {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

START WORD: SFOTTE
Finding levenshtein... (0.00)
('SFOTTA', 1) 12 levenshtein
Finding anagrams... (0.22)
('TASTO', 1) 15 anagram
Finding levenshtein... (0.01)
('PASTO', 1) 17 levenshtein
Finding levenshtein... (0.01)
('FASTO', 1) 20 levenshtein
Finding levenshtein... (0.01)
('FESTO', 1) 18 levenshtein
Finding anagrams... (0.12)
('SFOLTE', 1) 16 anagram
restart False

END WORD: SFOLTE
Elapsed 0.43s


In [171]:
def isInDictionary(word):
    results = search(word, 0)
    return len(results) > 0

class WordStatus():
  UNCHECKED = 0
  VALID = 1
  INVALID = 2
  NOT_IN_DICT = 3

def checkValidChain(startWord, endWord, wordChain):
    prevWord = startWord
    isOk = True
    statuses = []
    for word in wordChain + [endWord]:
        inDictionary = isInDictionary(word)
        results = [t[0] for t in search(prevWord, 1)]
        anagrams = [t[0] for t in findAnagramsEx(filteredWords, prevWord, maxDiffLength=1, strict=False)]
        isWordOk = inDictionary and (word in results or word in anagrams) 
        if not inDictionary:
            print("✖️", word)
            statuses.append(WordStatus.NOT_IN_DICT)
        else:
            print("✔️" if isWordOk else "❌", word)
            statuses.append(WordStatus.VALID if isWordOk else WordStatus.INVALID)
        
        if not isWordOk:
            isOk = False
        prevWord = word

    return isOk, statuses
            
print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: SFOTTE
END WORD  : SFOLTE


In [172]:
def displayCheckValidChain(startWord, endWord, wordChain):
    print()
    print(startWord)
    ok, statuses = checkValidChain(startWord, endWord, wordChain)
    print(endWord)

    print()
    print("👍 WON" if ok else "👎 LOST")

    print()


In [173]:
# regenerate start/end words
t0 = time.time()
steps = generateChain(quiet=True, startWord='piccolo')
print("\nGenerate {0:.2f}s".format(time.time() - t0))

startWord = steps[0]
endWord = steps[-1]

print("START WORD:", startWord)
print("END WORD  :", endWord)

START WORD: SCOZZI
restart False

END WORD: ZIRLANO

Generate 3.77s
START WORD: SCOZZI
END WORD  : ZIRLANO


In [174]:
wordChain = [
    'pantera',
    'pentola',
    'pantofola',
    'osteria',
    'ristorai',
    'piantala',
]
wordChain = [w.strip().upper() for w in wordChain]
#print("GUESS WORD CHAIN", wordChain)

t0 = time.time()
displayCheckValidChain(startWord, endWord, wordChain)
print("Validate {0:.2f}s".format(time.time() - t0))


SCOZZI
❌ PANTERA
❌ PENTOLA
❌ PANTOFOLA
❌ OSTERIA
❌ RISTORAI
❌ PIANTALA
❌ ZIRLANO
ZIRLANO

👎 LOST

Validate 4.89s


In [175]:
# STEPS

print("COMPUTED WORD CHAIN", steps)

COMPUTED WORD CHAIN ['SCOZZI', 'SOZZACI', 'CODAZZI', 'CAZZINO', 'POCANZI', 'PINZALO', 'ZIRLANO']


In [176]:
# find in dictionary

findWord = 'muovi'

search(findWord.strip().upper(), 0)


[('MUOVI', 0)]

In [177]:
print([hex(ord(ch)) for ch in "✖️"])
print([hex(ord(ch)) for ch in "✔️"])
print([hex(ord(ch)) for ch in "❌"])
"👍"
hex(ord("👎"))

['0x2716', '0xfe0f']
['0x2714', '0xfe0f']
['0x274c']


'0x1f44e'

In [178]:
@anvil.server.callable
def srvGenerateWordChain(quiet=True, startWord=None, strict=True):
  steps = generateChain(quiet=quiet, startWord=startWord, strict=strict)
  return steps

@anvil.server.callable
def srvValidateChain(startWord, endWord, wordChain):
  isOk, statuses = checkValidChain(startWord, endWord, wordChain)
  return isOk, statuses



In [179]:
t0 = time.time()

anagramsDict = {}
wordsByLength = {}
wordsBySortedAndLength = {}
for w in filteredWords[:]:
  sortedWord = ''.join(sorted(w))
  if sortedWord not in anagramsDict:
    anagramsDict[sortedWord] = set()
  anagramsDict[sortedWord].add(w)
  # moreAnagrams = findAnagramsEx(filteredWords, w, maxDiffLength=1, strict=True)
  # for item in moreAnagrams:
  #   anagramsDict[sortedWord].add(item[0])
  length = len(w)
  if length not in wordsByLength:
    wordsByLength[length] = set()
  wordsByLength[length].add(w)
  if w not in wordsBySortedAndLength:
    wordsBySortedAndLength[w] = [None, None, None]
  wordsBySortedAndLength[w][0] = sortedWord
  wordsBySortedAndLength[w][1] = length
  counter = Counter2(w)
  wordsBySortedAndLength[w][2] = counter


print(f'{(time.time() - t0):.2f}s')

display([item for item in random.choices(list(anagramsDict.items()), k=10)])
display([(k, list(v)[:5]) for k, v in wordsByLength.items()])
display([item for item in random.choices(list(wordsBySortedAndLength.items()), k=10)])


3.69s


[('ACDIIIMNR', {'DICRINAMI', 'INDICARMI'}),
 ('AACIINORV', {'RICONIAVA', 'VICARIANO'}),
 ('ADGIINNR', {'GINANDRI', 'GRANDINI', 'INDIGNAR', 'INGRANDI'}),
 ('AEIORSSSV', {'SVIASSERO', 'VISASSERO'}),
 ('AFIORRSS', {'SFORARSI'}),
 ('AILOPSSTT', {'PLOTTASSI'}),
 ('AAAEGLMNU', {'AUGNAMELA', 'MALAGUENA'}),
 ('AAAELLMRZ', {'ALZARMELA'}),
 ('AADGHIINR', {'DRAGHIANI'}),
 ('AGIILOSV', {'SVALIGIO', 'SVOGLIAI'})]

[(9, ['SVELTISCE', 'GIOVENCHI', 'FREEZARMI', 'TREMATAVI', 'GRUPPETTI']),
 (6, ['DURARE', 'REDINI', 'TASTAR', 'CAMINO', 'BUGNAI']),
 (5, ['LELLI', 'MERGI', 'VERVE', 'RONFI', 'GNOSI']),
 (7, ['PEDANTI', 'CAPARLO', 'BABISTA', 'SORANDO', 'ANIMOSI']),
 (8, ['GUSTEREI', 'IRRIGATI', 'OCCISESI', 'UNTARGLI', 'CERACENE']),
 (4, ['FUND', 'RECO', 'BEAR', 'OVAI', 'PLOT'])]

[('SORARSENE',
  ['AEENORRSS', 9, {'S': 2, 'O': 1, 'R': 2, 'A': 1, 'E': 2, 'N': 1}]),
 ('FORTIGNE',
  ['EFGINORT',
   8,
   {'F': 1, 'O': 1, 'R': 1, 'T': 1, 'I': 1, 'G': 1, 'N': 1, 'E': 1}]),
 ('COMPARALI',
  ['AACILMOPR',
   9,
   {'C': 1, 'O': 1, 'M': 1, 'P': 1, 'A': 2, 'R': 1, 'L': 1, 'I': 1}]),
 ('GROCCATE',
  ['ACCEGORT', 8, {'G': 1, 'R': 1, 'O': 1, 'C': 2, 'A': 1, 'T': 1, 'E': 1}]),
 ('GENARLE', ['AEEGLNR', 7, {'G': 1, 'E': 2, 'N': 1, 'A': 1, 'R': 1, 'L': 1}]),
 ('UNTALE', ['AELNTU', 6, {'U': 1, 'N': 1, 'T': 1, 'A': 1, 'L': 1, 'E': 1}]),
 ('BRUNETTA',
  ['ABENRTTU', 8, {'B': 1, 'R': 1, 'U': 1, 'N': 1, 'E': 1, 'T': 2, 'A': 1}]),
 ('CAZZACCIO', ['AACCCIOZZ', 9, {'C': 3, 'A': 2, 'Z': 2, 'I': 1, 'O': 1}]),
 ('IMPELARLE',
  ['AEEILLMPR', 9, {'I': 1, 'M': 1, 'P': 1, 'E': 2, 'L': 2, 'A': 1, 'R': 1}]),
 ('STAGNAMMO',
  ['AAGMMNOST', 9, {'S': 1, 'T': 1, 'A': 2, 'G': 1, 'N': 1, 'M': 2, 'O': 1}])]

In [180]:

# t0 = time.time()

# wordList = filteredWords[:21]
# for i, w in enumerate(wordList):
#   if i % 100 == 0:
#     print(f'{i:6}/{len(wordList)} {w}')
#   sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[w]
  
#   candidates = []
#   targetLength = wordLength - 1
#   if targetLength in wordsByLength:
#     candidates = list(wordsByLength[targetLength])
#   targetLength = wordLength + 1
#   if targetLength in wordsByLength:
#     candidates += list(wordsByLength[targetLength])
  
#   for targetWord in candidates:
#     sortedTargetWord, targetLength, targetCounter = wordsBySortedAndLength[targetWord]
#     isAnagram, diff = _isAnagramEx2(sortedWord, sortedTargetWord, wordLength, targetLength, maxDiffLength=1, strict=True)
    
#     # if len(w) < len(targetWord):
#     #   sourceCounter, targetCounter = targetCounter, sourceCounter
#     # diffCounter = sourceCounter - targetCounter
#     # diffSum = sum(diffCounter.values())
#     # isAnagram = diffSum == 1

#     #sourceCounter.isAnagram(targetCounter, maxDiffLength = 1)

#     if isAnagram:
#       # print("->", targetWord)
#       anagramsDict[sortedWord].add(targetWord)
#       # display(anagramsDict[sortedWord])

# print(f'{(time.time() - t0):.2f}s')


In [182]:
word = 'ABAVI'
sortedWord, wordLength, sourceCounter = wordsBySortedAndLength[word]
display(anagramsDict[sortedWord])

_isAnagramEx2(list('BAVA'), list('ABAVI'), 4, 5, maxDiffLength=1, strict=True)


{'ABAVI', 'BIAVA'}

(False, 2)

In [183]:
print('LUCRAMELO'.__hash__())
print('LUCRAMELO'.__hash__())
print('lucramelo'.__hash__())

for k,v in wordsByLength.items():
  print(k, len(v))

#findAnagramsEx(filteredWords, 'TAVOLIERE', maxDiffLength=1, strict=True)

1825648678846464053
1825648678846464053
5612121926921189805
9 200178
6 23116
5 9051
7 56496
8 109355
4 2927


[('AEROLITE', 1),
 ('AVERTELI', 1),
 ('AVERTELO', 1),
 ('AVERTILE', 1),
 ('AVERTILO', 1),
 ('AVOLTERI', 1),
 ('ELATERIO', 1),
 ('ELEVARTI', 1),
 ('ELEVATOR', 1),
 ('EVERTILA', 1),
 ('EVERTILO', 1),
 ('EVITARLE', 1),
 ('EVITARLO', 1),
 ('LEVATOIE', 1),
 ('LEVATORE', 1),
 ('LEVATORI', 1),
 ('LEVIRATO', 1),
 ('LEVITARE', 1),
 ('LEVITERA', 1),
 ('LEVITERO', 1),
 ('LIVERATE', 1),
 ('LIVERATO', 1),
 ('LIVREATE', 1),
 ('LIVREATO', 1),
 ('OLEATEVI', 1),
 ('ORLATEVI', 1),
 ('RELATIVE', 1),
 ('RELATIVO', 1),
 ('RILEVATE', 1),
 ('RILEVATO', 1),
 ('RIVALETE', 1),
 ('RIVELATE', 1),
 ('RIVELATO', 1),
 ('RIVOLATE', 1),
 ('RIVOLETE', 1),
 ('RIVOTALE', 1),
 ('VALITORE', 1),
 ('VETRIOLA', 1),
 ('VETRIOLE', 1),
 ('VIETARLE', 1),
 ('VIETARLO', 1),
 ('VIRATELE', 1),
 ('VIRATELO', 1),
 ('VIRATOLE', 1),
 ('VOIALTRE', 1),
 ('VOLITARE', 1),
 ('VOLITERA', 1),
 ('VOLTAIRE', 1),
 ('VOLTERAI', 1),
 ('VOLTEREI', 1),
 ('VORATELE', 1),
 ('VORATELI', 1)]

In [ ]:
anvil.server.wait_forever()